In [1]:
# MLP (Multilayer Perceptron)
# y = softmax(wx + b)
# Hidden layers를 추가한 것!!
# Activate function도 추가되었다.

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print ("CURRENT TF VERSION IS [%s]" % (tf.__version__))
print ("PACKAGES LOADED")

CURRENT TF VERSION IS [1.1.0]
PACKAGES LOADED


In [2]:
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# NETWORK TOPOLOGIES
n_input    = 784 
n_hidden_1 = 256 
n_hidden_2 = 128
n_classes  = 10

# INPUTS AND OUTPUTS
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])
    
# NETWORK PARAMETERS
stddev = 0.1
weights = {
    # 784 -> 256 dimension
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    # 256 -> 128 dimension
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    # 128 -> 10 dimension
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("NETWORK READY")

NETWORK READY


In [4]:
# MODEL

# placeholder로 입력(X)이 들어가고 weight와 bias가 들어가도록 되어 있다.
def multilayer_perceptron(_X, _weights, _biases):
    # 입력에 weight를 곱하고 bias를 더한 값에 
    # simoid(activate function)을 통과한 것이 나온다.
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    # Softmax가 없음, 아래 pred로 현재 함수를 통해 값을 구한 후
    # softmax_cross_entropy_with_logits를 사용할 것이기 때문
    return (tf.matmul(layer_2, _weights['out']) + _biases['out'])

# PREDICTION
pred = multilayer_perceptron(x, weights, biases)

# LOSS AND OPTIMIZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost) 
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


In [5]:
# PARAMETERS
training_epochs = 20
batch_size      = 100
display_step    = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
# OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y: batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY: %.3f" % (train_acc))
        feeds = {x: mnist.test.images, y: mnist.test.labels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
print ("OPTIMIZATION FINISHED")

Epoch: 003/020 cost: 0.121247095
TRAIN ACCURACY: 0.970
TEST ACCURACY: 0.965
Epoch: 007/020 cost: 0.048000447
TRAIN ACCURACY: 0.980
TEST ACCURACY: 0.977
Epoch: 011/020 cost: 0.018789678
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.979
Epoch: 015/020 cost: 0.006844028
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.979
Epoch: 019/020 cost: 0.002052205
TRAIN ACCURACY: 1.000
TEST ACCURACY: 0.981
OPTIMIZATION FINISHED
